In [295]:
import numpy as np
import pandas as pd

In [296]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [297]:
movies.shape

(4803, 20)

In [298]:
credits.shape

(4803, 4)

In [299]:
movies=movies.merge(credits,on='title')

In [300]:
movies.shape

(4809, 23)

In [301]:
movies = movies[['genres','id','keywords','title','overview','popularity','crew','release_date','cast']]

In [302]:
movies.dropna(inplace=True)

In [303]:
movies.duplicated().sum()

np.int64(0)

In [304]:
import ast

In [305]:
def separate(obj):
    list1=[]
    for i in ast.literal_eval(obj):
        list1.append(i['name'])
    return np.array(list1)

In [306]:
movies['genres']=movies['genres'].apply(separate)

In [307]:
movies['keywords']=movies['keywords'].apply(separate)

In [308]:
def covcast(obj):
    list1=[];
    c=0;
    for i in ast.literal_eval(obj):
        c+=1
        list1.append(i['name'])
        if(c==4): 
            break
    return np.array(list1)    

In [309]:
movies['cast']=movies['cast'].apply(covcast)

In [310]:
def convcrew(obj):
    list1=[]
    for i in ast.literal_eval(obj):
        if(i['job']=='Producer' or i['job']=='Director'):
            list1.append(i['name'])
    return list1


In [311]:
movies['crew']=movies['crew'].apply(convcrew)

In [312]:
def convrelease(obj):
    return obj[:4]

In [313]:
movies['release_date']=movies['release_date'].apply(convrelease)

In [314]:
def roundoff(num):
    num2=int(num)
    return ((num2+9)//10)*10

In [315]:
movies['popularity']=movies['popularity'].apply(roundoff)

In [316]:
movies['release_date']=movies['release_date'].apply(lambda x:round(int(x),-1))

In [317]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [318]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [319]:
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [320]:
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [321]:
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [322]:
movies['release_date']= movies['release_date'].apply(lambda x:[x]) 

In [323]:
movies['popularity']= movies['popularity'].apply(lambda x:[x]) 

In [324]:
movies['final']= movies['genres']+ movies['keywords']+movies['overview']+movies['popularity']+movies['crew']+movies['cast']+movies['release_date']

In [325]:
newdata=movies[['id','title','final']]

In [326]:
newdata['final'] =newdata['final'].apply(lambda x: " ".join(str(i) for i in x))

C:\Users\kgaks\AppData\Local\Temp\ipykernel_2780\3474125888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['final'] =newdata['final'].apply(lambda x: " ".join(str(i) for i in x))


In [327]:
newdata['final'] = newdata['final'].apply(lambda x: x.lower())

C:\Users\kgaks\AppData\Local\Temp\ipykernel_2780\4267607298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['final'] = newdata['final'].apply(lambda x: x.lower())


In [328]:
import nltk

In [329]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [330]:
def makesame(string):
    list1=[]
    for i in string.split():
        list1.append(ps.stem(i))
    return " ".join(list1)

In [331]:
newdata['final']=newdata['final'].apply(makesame)

C:\Users\kgaks\AppData\Local\Temp\ipykernel_2780\652859141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['final']=newdata['final'].apply(makesame)


In [332]:
from sklearn.feature_extraction.text import CountVectorizer

In [333]:
cv = CountVectorizer(max_features=6000, stop_words='english')

In [334]:
vectors=cv.fit_transform(newdata['final']).toarray()

In [335]:
from sklearn.metrics.pairwise import cosine_similarity

In [336]:
similarityvec2d=cosine_similarity(vectors)

In [337]:
def recommend(movie):
    m_index=newdata[newdata['title']==movie].index[0]
    distancevec=similarityvec2d[m_index]
    recommendindex=sorted(list(enumerate(distancevec)),reverse=True,key=lambda x:x[1])[1:6]
    for i in recommendindex:
        print(newdata['title'][i[0]])
    

In [339]:
recommend('Space Pirate Captain Harlock')

Meet Dave
Lake Mungo
Sardaarji
Treasure Planet
Ender's Game
